In [1]:
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import os
from keras.models import Model
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout, Input, AveragePooling2D, Lambda
from urllib import request
from keras.callbacks import TensorBoard
import tempfile
from keras import optimizers
from keras.utils import Sequence
import numpy as np
from tqdm import tqdm
from datetime import datetime
from keras.applications import vgg16, resnet50

Using TensorFlow backend.


In [2]:
NOT_KAGGLE_KERNEL = os.environ.get('NOT_KAGGLE_KERNEL', 'false') == 'true'
print('Run on kaggle kernel:', NOT_KAGGLE_KERNEL)

Run on kaggle kernel: True


In [3]:
ROOT_DIR = '../input/dog-breed' if NOT_KAGGLE_KERNEL else '../input'
EPOCHS = 100
IMAGE_SIZE = (224,224)
INPUT_SHAPE = IMAGE_SIZE + (3,)

In [4]:
def fetch_data():
    if not os.path.exists(ROOT_DIR):
        os.makedirs(ROOT_DIR)
        zip_path = os.path.join(tempfile.gettempdir(), 'dog-breed.zip')
        print('Start download!')
        request.urlretrieve('https://kienle.blob.core.windows.net/public/kaggle/dog-breed.zip', zip_path)
        print('Start unzip')
        import zipfile
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(ROOT_DIR)
        with zipfile.ZipFile(os.path.join(ROOT_DIR,'test.zip'), 'r') as zip_ref:
            zip_ref.extractall(ROOT_DIR)
        with zipfile.ZipFile(os.path.join(ROOT_DIR,'train.zip'), 'r') as zip_ref:
            zip_ref.extractall(ROOT_DIR)
        os.remove(zip_path)
        os.remove(os.path.join(ROOT_DIR,'train.zip'))
        os.remove(os.path.join(ROOT_DIR,'test.zip'))
        print('Done')
if(NOT_KAGGLE_KERNEL):
    fetch_data()

In [5]:
# Confirm all data are ready 
# Expected output: labels.csv  sample_submission.csv  test  train
!ls $ROOT_DIR

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
pd_images = pd.read_csv(os.path.join(ROOT_DIR, 'labels.csv'))

In [7]:
pd_images.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [8]:
train_valid_generator = ImageDataGenerator().flow_from_dataframe(pd_images, os.path.join(ROOT_DIR, 'train'), batch_size=32, x_col='id', y_col='breed', has_ext=False, target_size=IMAGE_SIZE)

Found 10222 images belonging to 120 classes.


In [9]:
num_classes = len(train_valid_generator.class_indices)

In [10]:
class FeaturesExtractor():
    def __init__(self, input_shape, pretrained_model, preprocess_input):
        self.input_shape = input_shape
        model = pretrained_model(include_top=False, input_shape=self.input_shape, weights='imagenet')
        inputs = Input(self.input_shape)
        x = inputs
        x = Lambda(preprocess_input, name='preprocessing')(x)
        x = model(x)
        x = GlobalAveragePooling2D()(x)
        model = Model(inputs, x)
        self.model = model
    def predict(self, X):
        return self.model.predict(X)

In [11]:
vgg_features_extractor = FeaturesExtractor(INPUT_SHAPE, vgg16.VGG16, vgg16.preprocess_input)
resnet50_features_extractor = FeaturesExtractor(INPUT_SHAPE, resnet50.ResNet50, resnet50.preprocess_input)

C:\Users\trule\AppData\Local\Continuum\anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [12]:
def create_simple_model(num_classes, input_shape):
    inputs = Input(input_shape)
    x = inputs
    x = Dropout(0.5)(x)
    x = Dense(num_classes, activation='softmax')(x)
    return Model(inputs, x)


In [13]:
X_train = np.zeros((train_valid_generator.n,) + INPUT_SHAPE)
y_train = np.zeros((train_valid_generator.n, 120))
for i in tqdm(range(len(train_valid_generator)), ncols=100):
    temp_x, temp_y = train_valid_generator[i]
    start_index = i*train_valid_generator.batch_size
    X_train[start_index:start_index + temp_x.shape[0]] = temp_x
    y_train[start_index:start_index + temp_x.shape[0]] = temp_y


100%|█████████████████████████████████████████████████████████████| 320/320 [00:43<00:00,  7.42it/s]


In [15]:
# Calculate vgg features
vgg_features = vgg_features_extractor.predict(X_train)

In [17]:
vgg_based_model = create_simple_model(num_classes, vgg_features_extractor.model.output.get_shape().as_list()[1:])
vgg_based_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
if NOT_KAGGLE_KERNEL:
    callbacks = [TensorBoard('./data/logs/vgg-{0}'.format(datetime.now().isoformat().replace(':','-')))]
else:
    callbacks = []
    
vgg_based_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               61560     
Total params: 61,560
Trainable params: 61,560
Non-trainable params: 0
_________________________________________________________________


In [18]:
vgg_based_model.fit(vgg_features, y_train, batch_size=256, epochs=EPOCHS, callbacks=callbacks, validation_split=0.1)

Train on 9199 samples, validate on 1023 samples
Epoch 1/100
9199/9199 [==============================] - ETA: 14s - loss: 15.1680 - acc: 0.011 - ETA: 1s - loss: 15.0745 - acc: 0.013 - ETA: 0s - loss: 14.9673 - acc: 0.01 - ETA: 0s - loss: 14.8332 - acc: 0.01 - 1s 82us/step - loss: 14.7462 - acc: 0.0208 - val_loss: 12.4536 - val_acc: 0.0655
Epoch 2/100
9199/9199 [==============================] - ETA: 0s - loss: 14.1425 - acc: 0.03 - ETA: 0s - loss: 14.1213 - acc: 0.03 - ETA: 0s - loss: 13.9854 - acc: 0.04 - ETA: 0s - loss: 13.8337 - acc: 0.04 - 0s 21us/step - loss: 13.7303 - acc: 0.0498 - val_loss: 10.6226 - val_acc: 0.1476
Epoch 3/100
9199/9199 [==============================] - ETA: 0s - loss: 12.7442 - acc: 0.07 - ETA: 0s - loss: 12.7943 - acc: 0.08 - ETA: 0s - loss: 12.4917 - acc: 0.09 - ETA: 0s - loss: 12.3515 - acc: 0.09 - 0s 20us/step - loss: 12.3414 - acc: 0.0994 - val_loss: 8.9972 - val_acc: 0.2317
Epoch 4/100
9199/9199 [==============================] - ETA: 0s - loss: 10.5868

9199/9199 [==============================] - ETA: 0s - loss: 2.6980 - acc: 0.652 - ETA: 0s - loss: 2.7642 - acc: 0.631 - ETA: 0s - loss: 2.7082 - acc: 0.633 - ETA: 0s - loss: 2.7267 - acc: 0.635 - 0s 19us/step - loss: 2.7273 - acc: 0.6346 - val_loss: 2.8446 - val_acc: 0.6022
Epoch 31/100
9199/9199 [==============================] - ETA: 0s - loss: 2.2740 - acc: 0.664 - ETA: 0s - loss: 2.6458 - acc: 0.629 - ETA: 0s - loss: 2.6227 - acc: 0.635 - 0s 18us/step - loss: 2.6836 - acc: 0.6340 - val_loss: 2.7049 - val_acc: 0.6364
Epoch 32/100
9199/9199 [==============================] - ETA: 0s - loss: 2.3172 - acc: 0.691 - ETA: 0s - loss: 2.6288 - acc: 0.639 - ETA: 0s - loss: 2.6561 - acc: 0.636 - 0s 17us/step - loss: 2.6814 - acc: 0.6377 - val_loss: 2.7499 - val_acc: 0.6266
Epoch 33/100
9199/9199 [==============================] - ETA: 0s - loss: 2.5859 - acc: 0.664 - ETA: 0s - loss: 2.7324 - acc: 0.633 - ETA: 0s - loss: 2.5601 - acc: 0.649 - ETA: 0s - loss: 2.5749 - acc: 0.647 - 0s 24us/step

Epoch 60/100
9199/9199 [==============================] - ETA: 0s - loss: 2.0693 - acc: 0.718 - ETA: 0s - loss: 2.0286 - acc: 0.686 - ETA: 0s - loss: 1.9939 - acc: 0.682 - ETA: 0s - loss: 2.0304 - acc: 0.679 - 0s 20us/step - loss: 2.0396 - acc: 0.6787 - val_loss: 2.4781 - val_acc: 0.6305
Epoch 61/100
9199/9199 [==============================] - ETA: 0s - loss: 1.8783 - acc: 0.668 - ETA: 0s - loss: 2.0478 - acc: 0.689 - ETA: 0s - loss: 2.0504 - acc: 0.686 - 0s 18us/step - loss: 2.0456 - acc: 0.6834 - val_loss: 2.4824 - val_acc: 0.6383
Epoch 62/100
9199/9199 [==============================] - ETA: 0s - loss: 2.1971 - acc: 0.722 - ETA: 0s - loss: 1.9834 - acc: 0.703 - ETA: 0s - loss: 2.0220 - acc: 0.691 - 0s 18us/step - loss: 2.0396 - acc: 0.6874 - val_loss: 2.4781 - val_acc: 0.6325
Epoch 63/100
9199/9199 [==============================] - ETA: 0s - loss: 2.0520 - acc: 0.691 - ETA: 0s - loss: 1.9874 - acc: 0.685 - ETA: 0s - loss: 2.0215 - acc: 0.685 - ETA: 0s - loss: 2.0154 - acc: 0.683 -

9199/9199 [==============================] - ETA: 0s - loss: 1.5459 - acc: 0.738 - ETA: 0s - loss: 1.6610 - acc: 0.708 - ETA: 0s - loss: 1.6603 - acc: 0.704 - 0s 18us/step - loss: 1.7084 - acc: 0.7016 - val_loss: 2.1520 - val_acc: 0.6520
Epoch 92/100
9199/9199 [==============================] - ETA: 0s - loss: 1.9458 - acc: 0.703 - ETA: 0s - loss: 1.7164 - acc: 0.699 - ETA: 0s - loss: 1.7420 - acc: 0.699 - 0s 18us/step - loss: 1.7206 - acc: 0.6984 - val_loss: 2.1641 - val_acc: 0.6569
Epoch 93/100
9199/9199 [==============================] - ETA: 0s - loss: 1.5598 - acc: 0.750 - ETA: 0s - loss: 1.6624 - acc: 0.717 - ETA: 0s - loss: 1.6967 - acc: 0.710 - 0s 18us/step - loss: 1.6991 - acc: 0.7084 - val_loss: 2.1809 - val_acc: 0.6500
Epoch 94/100
9199/9199 [==============================] - ETA: 0s - loss: 1.5986 - acc: 0.726 - ETA: 0s - loss: 1.6170 - acc: 0.721 - ETA: 0s - loss: 1.6839 - acc: 0.709 - 0s 18us/step - loss: 1.7164 - acc: 0.7051 - val_loss: 2.2461 - val_acc: 0.6422
Epoch 95/

In [19]:
resnet50_features = resnet50_features_extractor.predict(X_train)

In [20]:
resnet50_based_model = create_simple_model(num_classes, resnet50_features_extractor.model.output.get_shape().as_list()[1:])
resnet50_based_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
if NOT_KAGGLE_KERNEL:
    callbacks = [TensorBoard('./data/logs/resnet50-{0}'.format(datetime.now().isoformat().replace(':','-')))]
else:
    callbacks = []

In [21]:
resnet50_based_model.fit(resnet50_features, y_train, batch_size=256, epochs=EPOCHS,  callbacks=callbacks, validation_split=0.1)

Train on 9199 samples, validate on 1023 samples
Epoch 1/100
9199/9199 [==============================] - ETA: 6s - loss: 6.1272 - acc: 0.011 - ETA: 0s - loss: 5.5317 - acc: 0.023 - ETA: 0s - loss: 5.0511 - acc: 0.044 - ETA: 0s - loss: 4.5269 - acc: 0.094 - ETA: 0s - loss: 4.0939 - acc: 0.146 - 1s 63us/step - loss: 3.8938 - acc: 0.1713 - val_loss: 1.7258 - val_acc: 0.5963
Epoch 2/100
9199/9199 [==============================] - ETA: 0s - loss: 2.1010 - acc: 0.449 - ETA: 0s - loss: 1.9940 - acc: 0.474 - ETA: 0s - loss: 1.8797 - acc: 0.502 - ETA: 0s - loss: 1.7909 - acc: 0.524 - ETA: 0s - loss: 1.7185 - acc: 0.539 - 0s 27us/step - loss: 1.6979 - acc: 0.5452 - val_loss: 1.1000 - val_acc: 0.7038
Epoch 3/100
9199/9199 [==============================] - ETA: 0s - loss: 1.3650 - acc: 0.605 - ETA: 0s - loss: 1.2710 - acc: 0.631 - ETA: 0s - loss: 1.2229 - acc: 0.648 - ETA: 0s - loss: 1.2057 - acc: 0.656 - ETA: 0s - loss: 1.1790 - acc: 0.663 - 0s 27us/step - loss: 1.1742 - acc: 0.6647 - val_loss:

Epoch 26/100
9199/9199 [==============================] - ETA: 0s - loss: 0.1404 - acc: 0.976 - ETA: 0s - loss: 0.1574 - acc: 0.962 - ETA: 0s - loss: 0.1632 - acc: 0.959 - ETA: 0s - loss: 0.1658 - acc: 0.958 - ETA: 0s - loss: 0.1677 - acc: 0.957 - 0s 27us/step - loss: 0.1696 - acc: 0.9564 - val_loss: 0.7203 - val_acc: 0.7732
Epoch 27/100
9199/9199 [==============================] - ETA: 0s - loss: 0.1431 - acc: 0.972 - ETA: 0s - loss: 0.1657 - acc: 0.957 - ETA: 0s - loss: 0.1580 - acc: 0.961 - ETA: 0s - loss: 0.1592 - acc: 0.959 - ETA: 0s - loss: 0.1613 - acc: 0.959 - 0s 27us/step - loss: 0.1619 - acc: 0.9589 - val_loss: 0.7436 - val_acc: 0.7586
Epoch 28/100
9199/9199 [==============================] - ETA: 0s - loss: 0.1334 - acc: 0.968 - ETA: 0s - loss: 0.1471 - acc: 0.965 - ETA: 0s - loss: 0.1508 - acc: 0.962 - ETA: 0s - loss: 0.1532 - acc: 0.962 - ETA: 0s - loss: 0.1563 - acc: 0.961 - 0s 27us/step - loss: 0.1569 - acc: 0.9604 - val_loss: 0.7455 - val_acc: 0.7781
Epoch 29/100
9199/9

9199/9199 [==============================] - ETA: 0s - loss: 0.0922 - acc: 0.976 - ETA: 0s - loss: 0.0910 - acc: 0.976 - ETA: 0s - loss: 0.0909 - acc: 0.974 - ETA: 0s - loss: 0.0946 - acc: 0.972 - 0s 25us/step - loss: 0.0936 - acc: 0.9736 - val_loss: 0.8013 - val_acc: 0.7693
Epoch 52/100
9199/9199 [==============================] - ETA: 0s - loss: 0.0650 - acc: 0.988 - ETA: 0s - loss: 0.0801 - acc: 0.980 - ETA: 0s - loss: 0.0873 - acc: 0.977 - ETA: 0s - loss: 0.0903 - acc: 0.975 - 0s 26us/step - loss: 0.0904 - acc: 0.9758 - val_loss: 0.8135 - val_acc: 0.7713
Epoch 53/100
9199/9199 [==============================] - ETA: 0s - loss: 0.1078 - acc: 0.960 - ETA: 0s - loss: 0.0939 - acc: 0.976 - ETA: 0s - loss: 0.0903 - acc: 0.977 - ETA: 0s - loss: 0.0902 - acc: 0.977 - ETA: 0s - loss: 0.0893 - acc: 0.977 - 0s 28us/step - loss: 0.0891 - acc: 0.9777 - val_loss: 0.7986 - val_acc: 0.7693
Epoch 54/100
9199/9199 [==============================] - ETA: 0s - loss: 0.0708 - acc: 0.984 - ETA: 0s - lo

9199/9199 [==============================] - ETA: 0s - loss: 0.0609 - acc: 0.992 - ETA: 0s - loss: 0.0630 - acc: 0.983 - ETA: 0s - loss: 0.0628 - acc: 0.983 - ETA: 0s - loss: 0.0605 - acc: 0.984 - ETA: 0s - loss: 0.0604 - acc: 0.984 - 0s 27us/step - loss: 0.0605 - acc: 0.9843 - val_loss: 0.8639 - val_acc: 0.7664
Epoch 77/100
9199/9199 [==============================] - ETA: 0s - loss: 0.0553 - acc: 0.980 - ETA: 0s - loss: 0.0602 - acc: 0.982 - ETA: 0s - loss: 0.0632 - acc: 0.983 - ETA: 0s - loss: 0.0640 - acc: 0.982 - ETA: 0s - loss: 0.0648 - acc: 0.982 - 0s 27us/step - loss: 0.0644 - acc: 0.9827 - val_loss: 0.8365 - val_acc: 0.7771
Epoch 78/100
9199/9199 [==============================] - ETA: 0s - loss: 0.0412 - acc: 0.996 - ETA: 0s - loss: 0.0530 - acc: 0.989 - ETA: 0s - loss: 0.0546 - acc: 0.987 - ETA: 0s - loss: 0.0571 - acc: 0.986 - ETA: 0s - loss: 0.0572 - acc: 0.986 - 0s 28us/step - loss: 0.0586 - acc: 0.9852 - val_loss: 0.8481 - val_acc: 0.7781
Epoch 79/100
9199/9199 [========